In [30]:
import polars as pl
REGRESSION_TARGET = "prix"
CLASSIFICATION_TARGET = "en_dessous_du_marche"


In [4]:
# A REMPLACER EN FONCTION DE LA LOCALISATION DE VOTRE DONNEE DANS VOTRE DOSSIER
transactions = pl.read_parquet("transactions_immobilieres.parquet")


## AVDJ P1C2

On va se limiter au pérmiètre suivant pour ce exercice 

In [56]:
transactions_exercice = transactions.filter(
    pl.col("departement") == 75,
).select(["surface_habitable", "n_pieces", "prix", "vefa", "mois_transaction", "annee_transaction"])


X = transactions_exercice.drop("prix")
y = transactions_exercice["prix"]


In [57]:
import statsmodels.api as sm

On entraine la regression linéaire avec la syntaxe de statsmodels. Cette librairie n'est malheureusement pas compatible avec Polars, nous devons le convertir en DataFrame Pandas avec la méthode to_pandas()

In [59]:

linear_regressor = sm.OLS(endog = y.to_pandas(), exog = X.to_pandas()).fit()

La méthode summary() nous donne toutes les informations pertinentes sur la regression linéaire

In [60]:
linear_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   prix   R-squared (uncentered):                   0.819
Model:                            OLS   Adj. R-squared (uncentered):              0.819
Method:                 Least Squares   F-statistic:                          1.012e+05
Date:                Wed, 24 Jul 2024   Prob (F-statistic):                        0.00
Time:                        12:51:48   Log-Likelihood:                     -1.5588e+06
No. Observations:              111725   AIC:                                  3.118e+06
Df Residuals:                  111720   BIC:                                  3.118e+06
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
surface_habitable   1.12e+04     45.771    244.598      0.000    1.11e+04    1.13e+04
n_pieces          -1.295e+04   1308.434     -9.895      0.000   -1.55e+04   -1.04e+04
vefa              -1.709e+04    1.2e+04     -1.428      0.153   -4.06e+04    6369.925
mois_transaction    435.9821    241.770      1.803      0.071     -37.883     909.847
annee_transaction    -9.5656      1.157     -8.265      0.000     -11.834      -7.297
==============================================================================
Omnibus:                   137979.569   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        357047942.483
Skew:                           5.685   Prob(JB):                         0.00
Kurtosis:                     279.712   Cond. No.                     2.91e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Nous suivons le processus d'élimination backwards qui peut être résumé comme suit :
* On entraine un premier modèle avec toutes les features
* Si on a des features avec une p-valeur supérieure à 0.05, c'est que notre regression linéaire n'est pas fiable. Sinon, notre modèle est bon à être interprété et utilisé.
* Dans le premier cas, on identifie la feature avec la p-value la plus élevée. On la supprime et on réentraine le modèle.
* On répète ce processus jusqu'à l'obtention d'un assortiment de features avec des p-valeurs en dessous du seuil.

Concrétement : Présence de p-valeurs supérieurs au seuil signifie que les features n'ont pas de lien avéré statistiquement parlant avec la target, donc les coefficients associés aux features ne sont pas fiable. 

In [61]:
# Il faut supprimer la variable avec la P-valeur la plus élevée
X = X.drop("vefa")

Attention : Ce serait une erreur de supprimer TOUTES les features avec un p-value supérieure au seuil de 0.05 avant de réentrainer le 2eme modèle. En effet, en supprimant qu'une seule feature, les p-valeur vont bouger de nouveau après avoir réentrainé le modèle. Il se peut qu'une toute autre feature soit au-dessus de seuil, ou bien aucune.

In [63]:
linear_regressor = sm.OLS(endog = y.to_pandas(), exog = X.to_pandas()).fit()
linear_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   prix   R-squared (uncentered):                   0.819
Model:                            OLS   Adj. R-squared (uncentered):              0.819
Method:                 Least Squares   F-statistic:                          1.265e+05
Date:                Wed, 24 Jul 2024   Prob (F-statistic):                        0.00
Time:                        12:54:47   Log-Likelihood:                     -1.5588e+06
No. Observations:              111725   AIC:                                  3.118e+06
Df Residuals:                  111721   BIC:                                  3.118e+06
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
surface_habitable  1.119e+04     45.765    244.606      0.000    1.11e+04    1.13e+04
n_pieces          -1.293e+04   1308.369     -9.881      0.000   -1.55e+04   -1.04e+04
mois_transaction    433.8710    241.766      1.795      0.073     -39.987     907.729
annee_transaction    -9.5972      1.157     -8.294      0.000     -11.865      -7.329
==============================================================================
Omnibus:                   137994.453   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        357093737.441
Skew:                           5.686   Prob(JB):                         0.00
Kurtosis:                     279.729   Cond. No.                     3.19e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
# Nous avons une fois de plus une variable dont la p-valeur est supérieure au seuil
X = X.drop("mois_transaction")

In [67]:
linear_regressor = sm.OLS(endog = y.to_pandas(), exog = X.to_pandas()).fit()
linear_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   prix   R-squared (uncentered):                   0.819
Model:                            OLS   Adj. R-squared (uncentered):              0.819
Method:                 Least Squares   F-statistic:                          1.687e+05
Date:                Wed, 24 Jul 2024   Prob (F-statistic):                        0.00
Time:                        12:59:09   Log-Likelihood:                     -1.5588e+06
No. Observations:              111725   AIC:                                  3.118e+06
Df Residuals:                  111722   BIC:                                  3.118e+06
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
surface_habitable  1.119e+04     45.766    244.600      0.000    1.11e+04    1.13e+04
n_pieces          -1.293e+04   1308.382     -9.881      0.000   -1.55e+04   -1.04e+04
annee_transaction    -8.2345      0.873     -9.431      0.000      -9.946      -6.523
==============================================================================
Omnibus:                   138008.265   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        357098071.616
Skew:                           5.687   Prob(JB):                         0.00
Kurtosis:                     279.731   Cond. No.                     3.19e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 3.19e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Passons désormais à l'interprétation des coefficients ! En se basant sur le resultat de la méthode summary(), nous pouvons récupérer les coefficients de la regression et écrire notre fameuse moyenne pondérée qui approxime la cible :

**estimation_transaction = (11190 * surface_habitable) - ( 12930 * nombre_de_pieces) - (8.23 * annee_transaction)**

A partir de là, on peut commencer à décrire comment le modèle change de comportement en fonction de comment les features changent :     


*   A surface et année fixe, un appartement avec une pièce en plus couterait selon le modèle a peu près 13k€ en moins. Peut-être car nous avons choisi Paris comme département, où il y a une tension particuière sur les studios
* A année et nombre de pièces fixe, un appartement avec 5 m2 de plus vaudrait a peu près 56k€ de plus. Cela parait assez logique.
*   Vu la faible valeur du coefficient pour la feature année, on peut conclure que l'année n'a pas un impact enorme sur la prédiction du modèle. Peut-être car Paris est encore une fois une exception du marché immobilier Francais où les prix restent élevés quelque soit la situation.

Notez qu'on ne peut raisonner qu'avec une seule feature à la fois ! On ne peut donc pas détérminer l'impact de deux features en même temps avec cette approche (les coefficients ne se cumulent pas !). On appelle cela un raisonnement "toute chose égale par ailleurs".




Si vous êtes motivés, vous pouvez automatiser ce processus en écrivant une fonction qui utilise une boucle while ;)  